In [ ]:
import os
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import librosa.display
import IPython.display as ipd
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Loading CSV file of overlap chunk data path and classlabels

audioPath_labels = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Final_OverlapData_audioPath_classLabels.csv")
print(audioPath_labels)

                                                AudioPath ClassLabels
0       /content/drive/MyDrive/Colab Notebooks/Fall 20...      aldfly
1       /content/drive/MyDrive/Colab Notebooks/Fall 20...      aldfly
2       /content/drive/MyDrive/Colab Notebooks/Fall 20...      aldfly
3       /content/drive/MyDrive/Colab Notebooks/Fall 20...      aldfly
4       /content/drive/MyDrive/Colab Notebooks/Fall 20...      aldfly
...                                                   ...         ...
389773  /content/drive/MyDrive/Colab Notebooks/Fall 20...      yetvir
389774  /content/drive/MyDrive/Colab Notebooks/Fall 20...      yetvir
389775  /content/drive/MyDrive/Colab Notebooks/Fall 20...      yetvir
389776  /content/drive/MyDrive/Colab Notebooks/Fall 20...      yetvir
389777  /content/drive/MyDrive/Colab Notebooks/Fall 20...      yetvir

[389778 rows x 2 columns]


**Visual Feature - Mel-Spectogram Extraction**

In [ ]:
# Visual features for Overlap Data using abs() in power_db
errorFileId = []
vis_feat = np.zeros((10000, 128, 431), dtype = float)

noChunks = 10000
sr = 44100  

n_fft = 1024
hop_length = 512

print("Chunks of 10,000 :")

for i in range(0, len(audioPath_labels["AudioPath"])//noChunks):

  curChunk = 0
  for id in range(i*noChunks,(i+1)*noChunks):

    if (id%100 == 0 or id==(len(audioPath_labels)-1)):
      print("Processed Audios : ", id)

    try:
      x1, sr1 = librosa.load(audioPath_labels["AudioPath"][id], sr=sr)

      # Apply Mel-spec
      mel = librosa.feature.melspectrogram(x1, sr, n_mels=128, n_fft=1024, hop_length=512, fmin=20, fmax=16000)
      # transforming to mel scale
      mel_scale = librosa.power_to_db(abs(mel))

      # Append into .npy
      vis_feat[curChunk] = mel_scale

    except Exception as e:
      print(e)
      errorFileId.append(id)
      print("File Path: " + audioPath_labels["AudioPath"][id])

    curChunk = curChunk + 1

  #Saving file
  a1 = str(i*noChunks)
  a2 = str((i+1)*noChunks)
  filename = "vis_feat_"+a1+"_"+a2+".npy"
  np.save('/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap/'+filename, vis_feat)
  print("Saved : ", i*noChunks, " to ", (i+1)*noChunks)

Chunks of 10,000 :
Processed Audios :  360000
Processed Audios :  360100
Processed Audios :  360200
Processed Audios :  360300
Processed Audios :  360400
Processed Audios :  360500
Processed Audios :  360600
Processed Audios :  360700
Processed Audios :  360800
Processed Audios :  360900
Processed Audios :  361000
Processed Audios :  361100
Processed Audios :  361200
Processed Audios :  361300
Processed Audios :  361400
Processed Audios :  361500
Processed Audios :  361600
Processed Audios :  361700
Processed Audios :  361800
Processed Audios :  361900
Processed Audios :  362000
Processed Audios :  362100
Processed Audios :  362200
Processed Audios :  362300
Processed Audios :  362400
Processed Audios :  362500
Processed Audios :  362600
Processed Audios :  362700
Processed Audios :  362800
Processed Audios :  362900
Processed Audios :  363000
Processed Audios :  363100
Processed Audios :  363200
Processed Audios :  363300
Processed Audios :  363400
Processed Audios :  363500
Processed

**Converting Mel-Spectograms to RGB**

In [ ]:
# Loading 10000 visual features one by one and converting them into RGB
for k in range(40):
  filename = "vis_feat_"+str(k*10000)+"_"+str((k+1)*10000)+".npy"

  CBCData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap/"+filename)

  CBCDataRGB = np.zeros((5000,128,431,3), dtype='uint8')

  # Splitting 10000 visual features into two separate files of 5000 visual features
  for j in range(2):
    curChunk = 0

    # Converting visual features into RGB one by one 
    for i in range(j*len(CBCDataRGB), (j+1)*len(CBCDataRGB)):
      rgbData0 = np.stack([CBCData[i], CBCData[i], CBCData[i]], axis=-1)
      mean = rgbData0.mean()
      std = rgbData0.std()
      Xstd = (rgbData0 - mean) / (std + 1e-6)
      _min, _max = Xstd.min(), Xstd.max()
      norm_max = _max
      norm_min = _min
      if (_max - _min) > 1e-6:
          # Scale to [0, 255]
          V = Xstd
          V[V < norm_min] = norm_min
          V[V > norm_max] = norm_max
          V = 255 * (V - norm_min) / (norm_max - norm_min)
          CBCDataRGB[curChunk] = V
      else:
          # Just zero
          V = np.zeros_like(Xstd, dtype=np.uint8)
          CBCDataRGB[curChunk] = V
      curChunk = curChunk + 1

    # Saving file
    filename = "vis_feat_"+str((k*2)+j)+".npy"
    np.save('/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data/'+filename, CBCDataRGB)
    print("Saved : "+filename)

Saved : vis_feat_0.npy
Saved : vis_feat_1.npy
Saved : vis_feat_2.npy
Saved : vis_feat_3.npy
Saved : vis_feat_4.npy
Saved : vis_feat_5.npy
Saved : vis_feat_6.npy
Saved : vis_feat_7.npy
Saved : vis_feat_8.npy
Saved : vis_feat_9.npy
Saved : vis_feat_10.npy
Saved : vis_feat_11.npy
Saved : vis_feat_12.npy
Saved : vis_feat_13.npy
Saved : vis_feat_14.npy
Saved : vis_feat_15.npy
Saved : vis_feat_16.npy
Saved : vis_feat_17.npy
Saved : vis_feat_18.npy
Saved : vis_feat_19.npy
Saved : vis_feat_20.npy
Saved : vis_feat_21.npy
Saved : vis_feat_22.npy
Saved : vis_feat_23.npy
Saved : vis_feat_24.npy
Saved : vis_feat_25.npy
Saved : vis_feat_26.npy
Saved : vis_feat_27.npy
Saved : vis_feat_28.npy
Saved : vis_feat_29.npy
Saved : vis_feat_30.npy
Saved : vis_feat_31.npy
Saved : vis_feat_32.npy
Saved : vis_feat_33.npy
Saved : vis_feat_34.npy
Saved : vis_feat_35.npy
Saved : vis_feat_36.npy
Saved : vis_feat_37.npy
Saved : vis_feat_38.npy
Saved : vis_feat_39.npy
Saved : vis_feat_40.npy
Saved : vis_feat_41.npy
Sa

**One-Hot Encoding and Save labels**

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

labels = audioPath_labels['ClassLabels']

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [ ]:
# Save one-hot encoded labels

for i in range((len(audioPath_labels)//5000)+1):
  if(i == 77):
    filename = "labels_"+str(i)+".npy"
    np.save('/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_Overlap_RGB/Labels/'+filename, onehot_encoded[i*5000:])
    np.save('/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Labels/'+filename, onehot_encoded[i*5000:])
    print("Saved : "+filename)
  else:
    filename = "labels_"+str(i)+".npy"
    np.save('/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_Overlap_RGB/Labels/'+filename, onehot_encoded[i*5000:(i+1)*5000])
    np.save('/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Labels/'+filename, onehot_encoded[i*5000:(i+1)*5000])
    print("Saved : "+filename)

Saved : labels_0.npy
Saved : labels_1.npy
Saved : labels_2.npy
Saved : labels_3.npy
Saved : labels_4.npy
Saved : labels_5.npy
Saved : labels_6.npy
Saved : labels_7.npy
Saved : labels_8.npy
Saved : labels_9.npy
Saved : labels_10.npy
Saved : labels_11.npy
Saved : labels_12.npy
Saved : labels_13.npy
Saved : labels_14.npy
Saved : labels_15.npy
Saved : labels_16.npy
Saved : labels_17.npy
Saved : labels_18.npy
Saved : labels_19.npy
Saved : labels_20.npy
Saved : labels_21.npy
Saved : labels_22.npy
Saved : labels_23.npy
Saved : labels_24.npy
Saved : labels_25.npy
Saved : labels_26.npy
Saved : labels_27.npy
Saved : labels_28.npy
Saved : labels_29.npy
Saved : labels_30.npy
Saved : labels_31.npy
Saved : labels_32.npy
Saved : labels_33.npy
Saved : labels_34.npy
Saved : labels_35.npy
Saved : labels_36.npy
Saved : labels_37.npy
Saved : labels_38.npy
Saved : labels_39.npy
Saved : labels_40.npy
Saved : labels_41.npy
Saved : labels_42.npy
Saved : labels_43.npy
Saved : labels_44.npy
Saved : labels_45.np